Let's say that we've gotten a dataset that we'd like to benchmark for active learning. 

In [1]:
from sklearn.datasets import make_classification

X, y = make_classification(n_samples=100_000, n_classes=4, n_informative=10)

We will use modAL to handle some of the heavy liften. That means that we also need to write a compatible sampler if we want to have the random benchmark around.

In [2]:
import numpy as np
from modAL.uncertainty import entropy_sampling
from sklearn.linear_model import LogisticRegression

def randomly(classifier, X, n_instances=1):
    idx =  np.random.randint(0, X.shape[0] - 1, n_instances)
    return idx, X

We can now use the `run_experiment` from the `skteach.py` script to automate the data collection.

In [3]:
from skteach import run_experiment

In [4]:
?run_experiment

Signature:
run_experiment(
    name,
    data,
    estimator,
    strategy,
    batch_size=100,
    n_batch=10,
    start_size=50,
    idx_start=None,
)
Docstring:
Runs an active learning experiment and returns a list of results for a single run. 

Arguments:
    - name: the name for the experiment
    - data: the (X, y) data pair
    - estimator: the ML model to use
    - strategy: a modAL compatible selection function
    - batch_size: size of each labelling batch
    - n_batch: number of batches to run
    - start_size: if no `idx_start` are passed, how big the starting batch should be
    - idx_start: manually passed indices of the starting batch
File:      ~/Desktop/scikit-teach/skteach.py
Type:      function


In [5]:
data = []
for i in range(5): 
    data += run_experiment("entropy", data=(X, y), estimator=LogisticRegression(), strategy=entropy_sampling, n_batch=50)
for i in range(5):
    data += run_experiment("randomly", data=(X, y), estimator=LogisticRegression(), strategy=randomly, n_batch=50)

/usr/local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
100%|███████████████████████████████████| 50/50 [00:03<00:00, 13.43it/s]
/usr/local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.o

We can compare the scores with an altair chart now.

In [6]:
import pandas as pd
import altair as alt

pltr = pd.DataFrame(data)

(alt.Chart(pltr)
  .mark_line()
  .encode(x='batch', y='score', color='name', detail='run_id')
  .properties(width=600, height=250)
  .interactive())

alt.Chart(...)

Lesson one in active learning: random sampling is a suprisingly strong benchmark

In [7]:
from sklearn.ensemble import RandomForestClassifier

In [8]:
data = []
for i in range(5): 
    data += run_experiment("entropy", data=(X, y), estimator=RandomForestClassifier(), strategy=entropy_sampling, n_batch=50)
for i in range(5):
    data += run_experiment("randomly", data=(X, y), estimator=RandomForestClassifier(), strategy=randomly, n_batch=50)

100%|███████████████████████████████████| 50/50 [00:41<00:00,  1.20it/s]


In [9]:
pltr = pd.DataFrame(data)

(alt.Chart(pltr)
  .mark_line()
  .encode(x='batch', y='score', color='name', detail='run_id')
  .properties(width=600, height=250)
  .interactive())

alt.Chart(...)

In [ ]:


data = []
for i in range(5): 
    data += run_experiment("entropy", data=(X, y), estimator=GaussianProcessClassifier(), strategy=entropy_sampling, n_batch=50)
for i in range(5):
    data += run_experiment("randomly", data=(X, y), estimator=GaussianProcessClassifier(), strategy=randomly, n_batch=50)